<a href="https://colab.research.google.com/github/neerabhi/NLP/blob/main/Fake_new_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy

In [2]:
df=pd.read_csv('/content/Fake_Real_Data.csv')

In [3]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [6]:
df['label1']=df['label'].map({
    'Fake':0, 'Real':1
})

In [7]:
df.head()

,Text,label,label1
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [11]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
nlp=spacy.load('en_core_web_lg')

In [13]:
df['vector']=df['Text'].apply(lambda x: nlp(x).vector)

In [14]:
df.head()

,Text,label,label1,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.103623025, 0.17802684, -0.11873861, -0.034..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-0.0063406364, 0.16712041, -0.06661373, 0.017..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-0.122753024, 0.17192385, -0.024732638, -0.06..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-0.027337318, 0.12501417, -0.0073965387, -0.0..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-0.032708026, 0.093958504, -0.03287002, -0.00..."


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df['vector'], df['label1'], test_size=0.2)


In [17]:
X_train_2D=np.stack(X_train)
X_test_2D=np.stack(X_test)

In [18]:
X_train_2D.shape

(7920, 300)

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled_train=scaler.fit_transform(X_train_2D)
scaled_test=scaler.fit(X_test_2D)

In [24]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(scaled_train, y_train)


MultinomialNB()

In [28]:
y_pred=clf.predict(X_test_2D)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82      1000
           1       1.00      0.56      0.72       980

    accuracy                           0.78      1980
   macro avg       0.85      0.78      0.77      1980
weighted avg       0.85      0.78      0.77      1980



In [30]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)

In [31]:
knn.fit(scaled_train, y_train)
y_pred=knn.predict(X_test_2D)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.10      0.18      1000
           1       0.52      1.00      0.69       980

    accuracy                           0.55      1980
   macro avg       0.76      0.55      0.43      1980
weighted avg       0.76      0.55      0.43      1980

